In [13]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np 
from IPython.display import clear_output 
from six.moves import urllib
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc



In [14]:
dftrain = pd.read_csv('G:/projects/tensorflow/data/train.csv') #importing training data
dfeval = pd.read_csv('G:/projects/tensorflow/data/eval.csv')   #importing testing data
# print(dftrain.head())    #prints the first 5 elements of the dftrain dataset
# print(dfeval.head())     #prints the first 5 values of the dfeval dataset
y_train = dftrain.pop('survived')     #this statemnet pops the survived column from dftrain dataset and store it in y_train variable which also is our training label
y_eval = dfeval.pop('survived')       #this statemnet pops the survived column from dfeval dataset and store it in y_eval variable which also is our testing label

In [15]:
# print(dftrain.loc[0],y_train.loc[0]) #loc prints the rows which ever you wish to such as we print a element in array
# print(dftrain["age"])  #printing a specific column we need to name the column name and we are good to go 
# dftrain.head() #prints the first 5 entries of the dataset
# dftrain.describe()      # this describes the number of entries and their mean standard deviation and all that stuff


In [16]:
# dftrain.age.hist(bins=20) # ploting a histogram which shows 20 elements
# dftrain.sex.value_counts().plot(kind='barh')  # this statement prints the value of male and female

In [17]:
# dftrain["class"].value_counts().plot(kind ='barh')

In [18]:
# pd.concat([dftrain,y_train],axis =1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive') #this tells the survial rate according to whatever standard you choose her i chose class and age 
# dftrain.head()

In [63]:



feature_columns = []
CATEGORICAL_COLUMNS = ['class','deck','embark_town','alone','n_siblings_spouses','parch','sex']
NUMERIC_COLUMNS = ['fare','age']
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique() #gets a list of all unique values from given feature column
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name , vocabulary))
for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name , dtype=tf.float32))

# print(feature_columns) 
# dftrain['embark_town'].unique() # points out the unique value from whichever column we want to

In [20]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
    def input_function(): # inner fucntion, this willl be returned
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))  #create tf object with data and its label
        if shuffle:
            ds = ds.shuffle(1000) # randomizes order of data
        ds = ds.batch(batch_size).repeat(num_epochs) #split dataset into batches of 32 and repeat process for number of epochs
        return ds #return a batch of the dataset
    return input_function # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  #function call 
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs =1, shuffle = False)

In [64]:
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns) 
#we create a linear estimator by passing a feature_columns as argument

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ajaym\\AppData\\Local\\Temp\\tmp8y7wp11h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
linear_est.train(train_input_fn)  #training
result = linear_est.evaluate(eval_input_fn) #stores the stats after evaluating/ testing the model  

clear_output()
print(result['accuracy'])

0.78409094


In [62]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[14],y_eval.loc[14])
def result_function(percentage):
    if percentage>50.:
        print("survival chance : 1")
    else:
        print("survival chance : 0")
result_function((result[14]['probabilities'][1])*100)
    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ajaym\AppData\Local\Temp\tmppjz8p_0k\model.ckpt-1400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                        female
age                          29.0
n_siblings_spouses              0
parch                           0
fare                         10.5
class                      Second
deck                            F
embark_town           Southampton
alone                           y
Name: 14, dtype: object 1
survival chance : 1
